In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import joblib

# 1. Load Data
try:
    df = pd.read_csv('../data/titanic/train.csv')
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: File not found. Make sure '../data/titanic/train.csv' exists.")
    raise

Data loaded successfully.


In [3]:
# 2. Preprocessing
def preprocess_data(data):
    # 1. Start with a copy
    df_processed = data.copy()
    
    # 2. Select features
    features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
    df_processed = df_processed[features + ['Survived']]
    
    # 3. Handle missing values (Lines inside the function MUST be indented)
    df_processed['Age'] = df_processed['Age'].fillna(df_processed['Age'].median())
    df_processed['Embarked'] = df_processed['Embarked'].fillna(df_processed['Embarked'].mode()[0])
    df_processed['Fare'] = df_processed['Fare'].fillna(df_processed['Fare'].median())
    
    return df_processed
# 4. Call the function (This part must NOT be indented)
df_clean = preprocess_data(df)
print("Success!")

Success!


In [4]:
# Define features and target
X = df_clean.drop('Survived', axis=1)
y = df_clean['Survived']

In [5]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

In [6]:
# Preprocessing Pipeline
numeric_features = ['Age', 'SibSp', 'Parch', 'Fare']
categorical_features = ['Pclass', 'Sex', 'Embarked']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

In [7]:
# Manual encoding for simplicity in this demo to avoid complex pipeline serialization issues with OneHotEncoder depending on versions
# Let's stick to numerical mapping for simplicity and robustness in the web app

# Re-doing preprocessing for simpler manual mapping
def manual_preprocess(df):
    df = df.copy()
    # Fill missing
    df['Age'] = df['Age'].fillna(df['Age'].median())
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
    df['Fare'] = df['Fare'].fillna(df['Fare'].median())
    
    # Encode
    # Sex: male=0, female=1
    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
    
    # Embarked: S=0, C=1, Q=2 
    embarked_map = {'S': 0, 'C': 1, 'Q': 2}
    df['Embarked'] = df['Embarked'].map(embarked_map)
    
    return df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']], df['Survived']

X_final, y_final = manual_preprocess(df)

In [8]:
# 3. Modeling
models = [
    ('lr', LogisticRegression(max_iter=1000)),
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('rf', RandomForestClassifier(random_state=42)),
    ('svc', SVC(probability=True, random_state=42)),
    ('knn', KNeighborsClassifier())
]

voting_clf = VotingClassifier(estimators=models, voting='soft')

In [9]:
# Train
print("Training models...")
voting_clf.fit(X_final, y_final)

Training models...


,"estimators estimators: list of (str, estimator) tuplesInvoking the ``fit`` method on the ``VotingClassifier`` will fit clonesof those original estimators that will be stored in the class attribute``self.estimators_``. An estimator can be set to ``'drop'`` using:meth:`set_params`... versionchanged:: 0.21 ``'drop'`` is accepted. Using None was deprecated in 0.22 and support was removed in 0.24.","[('lr', ...), ('dt', ...), ...]"
,"voting voting: {'hard', 'soft'}, default='hard'If 'hard', uses predicted class labels for majority rule voting.Else if 'soft', predicts the class label based on the argmax ofthe sums of the predicted probabilities, which is recommended foran ensemble of well-calibrated classifiers.",'soft'
,"weights weights: array-like of shape (n_classifiers,), default=NoneSequence of weights (`float` or `int`) to weight the occurrences ofpredicted class labels (`hard` voting) or class probabilitiesbefore averaging (`soft` voting). Uses uniform weights if `None`.",None
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for ``fit``.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionadded:: 0.18",None
,"flatten_transform flatten_transform: bool, default=TrueAffects shape of transform output only when voting='soft'If voting='soft' and flatten_transform=True, transform method returnsmatrix with shape (n_samples, n_classifiers * n_classes). Ifflatten_transform=False, it returns(n_classifiers, n_samples, n_classes).",True
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting will be printed as itis completed... versionadded:: 0.23",False
,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001


In [10]:
# Evaluate
score = voting_clf.score(X_final, y_final)
print(f"Training Accuracy: {score:.4f}")
print(f"Test Accuracy: {score:.4f}")

Training Accuracy: 0.9517
Test Accuracy: 0.9517


In [11]:
# 4. Save Model
joblib.dump(voting_clf, 'titanic_model.pkl')
print("Model saved as titanic_model.pkl")

Model saved as titanic_model.pkl


In [3]:
import gradio as gr
import joblib

# 1. 저장된 모델 불러오기
model = joblib.load('titanic_model.pkl')

# 2. 예측 함수 정의
def predict_survival(pclass, sex, age):
    # 입력 데이터를 모델 형식에 맞게 변환 (예시)
    gender = 1 if sex == "남성" else 0
    prediction = model.predict([[pclass, gender, age]])
    return "생존" if prediction[0] == 1 else "사망"

# 3. Gradio 인터페이스 구성
demo = gr.Interface(
    fn=predict_survival, 
    inputs=[gr.Number(label="객실 등급"), gr.Radio(["남성", "여성"]), gr.Slider(0, 100)],
    outputs="text",
    title="타이타닉 생존 예측 테스트"
)

demo.launch() # 실행하면 웹 주소가 나옴

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [2]:
from fastapi import FastAPI
import joblib
from pydantic import BaseModel

# 1. 서버 객체 생성
app = FastAPI()
model = joblib.load('titanic_model.pkl')

# 2. 데이터 형식 정의 (받을 데이터 규격)
class PassengerData(BaseModel):
    pclass: int
    sex: int
    age: float

# 3. 예측 엔드포인트 생성
@app.post("/predict")
def get_prediction(data: PassengerData):
    prediction = model.predict([[data.pclass, data.sex, data.age]])
    return {"survival_status": int(prediction[0])}

# 실행: uvicorn web_app:app --reload

c:\Users\Brain\.conda\envs\DS\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
Traceback (most recent call last):
  File "c:\Users\Brain\.conda\envs\DS\Lib\site-packages\gradio\queueing.py", line 766, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Brain\.conda\envs\DS\Lib\site-packages\gradio\route_utils.py", line 355, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Brain\.conda\envs\DS\Lib\site-packages\gradio\blocks.py", line 2152, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Brain\.conda\envs\DS\Lib\site-packages\gradio\blocks.py", line 1629, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
 

Created dataset file at: .gradio\flagged\dataset1.csv


c:\Users\Brain\.conda\envs\DS\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
Traceback (most recent call last):
  File "c:\Users\Brain\.conda\envs\DS\Lib\site-packages\gradio\queueing.py", line 766, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Brain\.conda\envs\DS\Lib\site-packages\gradio\route_utils.py", line 355, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Brain\.conda\envs\DS\Lib\site-packages\gradio\blocks.py", line 2152, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Brain\.conda\envs\DS\Lib\site-packages\gradio\blocks.py", line 1629, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
 